In [1]:
from music21 import *
import os
import sklearn
import pandas as pd
import collections
import sys
import math
import numpy as np
import random

In [2]:
#TODO 
#Clustering 
#Combine

In [3]:
print(sys.path)

['', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python35.zip', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/plat-darwin', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/lib-dynload', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/extensions', '/Users/Dixit/.ipython']


In [4]:
#read midiToNotes.txt
notes  = pd.read_csv('midiToNotes.txt',skiprows=2)

In [ ]:
notes.sort_values(by='Offset').head(10)


In [6]:
notes2 = notes[:]

In [ ]:
notes2.sort_values(by='Offset').head(10)

In [8]:
actualNotes = ["%s%s" % (row[1]["Note/Rest"], row[1]["Octave"]) for row in notes2.iterrows()]


In [9]:
def createTuples(notes,n):
    for i,value in enumerate(notes):
        yield tuple(notes[i:i+n])

In [10]:
bigrams = collections.defaultdict(int);
trigrams = collections.defaultdict(int);
quadgrams = collections.defaultdict(int);

for i in createTuples(actualNotes,2):
    bigrams[i]+=1

for i in createTuples(actualNotes,3):
    trigrams[i]+=1

for i in createTuples(actualNotes,4):
    quadgrams[i]+=1
bigrams = dict((k, v) for k, v in bigrams.items() if v !=0 and len(k)==2)
trigrams = dict((k, v) for k, v in trigrams.items() if v!=0 and len(k)==3)
quadgrams = dict((k, v) for k, v in quadgrams.items() if v !=0 and len(k)==4)

In [11]:
#create a probability table based on above 
# eg for a bigram : 
#P(note-i | note-(i-1)) = count(note(i-1),note-i) / count(note(i-1))

def probTable(ngrams,prefixGrams):
    prbTable = collections.defaultdict(int);
    prevNgramDict=collections.defaultdict(list);
    for key,value in ngrams.items():
        prevngram  = key[:-1];
        currentNote = key[-1];
        #TODO some dont exist ; need to modify later ; TODO
        if prefixGrams[prevngram] !=0 :
            prbTable[(currentNote,prevngram)] = (float( ngrams[key] ) / prefixGrams[prevngram])
            #adding a new gram if doesnt exist; else extending existing one
            if prevngram not in prevNgramDict.keys():
                prevNgramDict[prevngram].extend([(currentNote, (float(ngrams[key]) / prefixGrams[prevngram]))])
                continue
            prevNgramDict[prevngram].extend([(currentNote, (float(ngrams[key]) / prefixGrams[prevngram]))])
    return prbTable,prevNgramDict
#prbTb,trigramDict     = probTable(quadgrams,trigrams)

triprobTb, bigramDict = probTable(trigrams, bigrams)


        


In [12]:
offsetList = [float(i) for ix,i in enumerate(notes["Offset"])]        


In [13]:
'''getting all offsets and counting their frequencies'''

def get2Offsets(offsetList):
    for idx,i in enumerate(offsetList):
        if idx == len(offsetList)-1:break
        yield offsetList[idx],offsetList[idx+1]

offsets = collections.defaultdict(int);

for a,b in get2Offsets(offsetList):
    diff = float(b) - float(a)
    diff = (math.ceil(diff*1000)/1000)
    if diff <4: #measure length
        offsets[diff]+=1
        
indexOffset = [k for k in offsets]
freqOffset = [offsets[k] for k in offsets]


In [14]:
#pruning of offsets and deelete
delOffsets = [jx for jx, j in enumerate(freqOffset) if j < 5 and (indexOffset[jx] < 2)]
indexOffset = [i for ix, i in enumerate(indexOffset) if ix not in delOffsets]
freqOffset = [j for jx, j in enumerate(freqOffset) if jx not in delOffsets]


In [15]:
len(freqOffset)

44

In [16]:
def normalize(freqOffset):
    sum = 0 
    norm = []
    for ix,i in enumerate(freqOffset):
        sum+=i
    for ix,i in enumerate(freqOffset):
        norm.append((float(i)/sum*1.0)*1);
    return norm

norm = normalize(freqOffset)


In [49]:
'''
- from bigram find all possible next notes and their probabilites
- check if all probabilities sum up to 1 or else normalize
- also see if next note is not same as previous notes
- finally randomly select based on probabilites
- do this for maybe 100 times

'''
# TODO improvements for octave changes to be done
def findNextNote(bigramDict,note1,note2):
    key  = (note1,note2);
    possibleNotes = np.array(bigramDict[key]);
    noteList = possibleNotes[:,0]
    frequencies = possibleNotes[:,1]
    frequencies = [float(f) for f in frequencies]
    totalprob = 0
    for f in frequencies:totalprob+=float(f)
    if totalprob!=1:
        frequencies = normalize(frequencies)
    return np.random.choice(noteList,p=frequencies)
    

def looper(bigramDict):
    keys = list(bigramDict)
    note1,note2 = keys[0]
    note3 = note2
    for i in range(600):
        note3 = findNextNote(bigramDict,note1,note2)
        note1 = note2
        note2 = note3
        yield note3

In [50]:
total = 600
done = 0
while total>done:
    try :
        newnotes = list(n for n in looper(bigramDict))
        freqOffsetNorm = normalize(freqOffset)
        newoffsets = list(np.random.choice(indexOffset, p=freqOffsetNorm) for i in range(len(newnotes)))
    except IndexError: 
        continue
    done = len(newnotes)

In [51]:
newnotes[0:10]

['C6', 'A5', 'F5', 'E-6', 'D6', 'B5', 'G#5', 'C6', 'G#5', 'F5']

In [21]:
bigramdict

defaultdict(list,
            {('A2', 'F2'): [('E-3', 1.0)],
             ('A3', 'A3'): [('E3', 1.0)],
             ('A3', 'B-4'): [('A4', 1.0)],
             ('A3', 'C4'): [('C#4', 0.25),
              ('G#3', 0.25),
              ('F3', 0.25),
              ('F4', 0.25)],
             ('A3', 'C5'): [('B-4', 1.0)],
             ('A3', 'C6'): [('D6', 1.0)],
             ('A3', 'D4'): [('F#3', 1.0)],
             ('A3', 'D6'): [('C6', 1.0)],
             ('A3', 'E-4'): [('A4', 1.0)],
             ('A3', 'E-5'): [('C#5', 1.0)],
             ('A3', 'E3'): [('E2', 0.3333333333333333),
              ('A3', 0.3333333333333333),
              ('C6', 0.3333333333333333)],
             ('A3', 'E5'): [('C#4', 1.0)],
             ('A3', 'F#2'): [('E3', 0.5), ('B-3', 0.5)],
             ('A3', 'F3'): [('G3', 1.0)],
             ('A3', 'F4'): [('G4', 0.5), ('B2', 0.5)],
             ('A3', 'F5'): [('G#4', 1.0)],
             ('A3', 'G#3'): [('F4', 1.0)],
             ('A3', 'G#5'): [('G#5', 1.0)],


In [52]:
with open("trigramNotes.txt", 'w') as f:
    for newnote, offset in zip(newnotes, newoffsets):
        f.write("%s,%s\n" % (newnote, offset))


In [53]:
newnotes

['C6',
 'A5',
 'F5',
 'E-6',
 'D6',
 'B5',
 'G#5',
 'C6',
 'G#5',
 'F5',
 'D5',
 'B4',
 'G#4',
 'F4',
 'B-1',
 'E4',
 'G#6',
 'G#5',
 'G#5',
 'D5',
 'C5',
 'G4',
 'D4',
 'B3',
 'F4',
 'F3',
 'G#3',
 'A3',
 'C4',
 'G#3',
 'G3',
 'E6',
 'F5',
 'B-5',
 'C6',
 'F5',
 'D3',
 'D6',
 'C4',
 'G#3',
 'D4',
 'G4',
 'A5',
 'B-4',
 'F#2',
 'G3',
 'G#3',
 'A3',
 'F4',
 'G4',
 'E4',
 'F4',
 'B-4',
 'A4',
 'F4',
 'G4',
 'C5',
 'C5',
 'A4',
 'A3',
 'D4',
 'F#3',
 'F#3',
 'C5',
 'D5',
 'A4',
 'A4',
 'D2',
 'D3',
 'C4',
 'D4',
 'E-4',
 'F4',
 'G4',
 'G3',
 'F#2',
 'B-3',
 'E-6',
 'B3',
 'D3',
 'C5',
 'B-5',
 'D5',
 'B-4',
 'G#4',
 'A4',
 'C5',
 'A4',
 'G4',
 'F4',
 'D4',
 'A3',
 'F3',
 'G3',
 'G#3',
 'A3',
 'C4',
 'F4',
 'A3',
 'F4',
 'B2',
 'A3',
 'C5',
 'B-4',
 'G4',
 'F4',
 'D4',
 'C4',
 'B3',
 'E4',
 'C4',
 'C6',
 'A5',
 'F5',
 'D5',
 'G#4',
 'F4',
 'D5',
 'F3',
 'F5',
 'G5',
 'F5',
 'C3',
 'B4',
 'B-3',
 'E-6',
 'D6',
 'G5',
 'G#5',
 'A5',
 'F#6',
 'C#6',
 'D6',
 'D5',
 'F5',
 'F5',
 'G5',
 'C5',
 